In [83]:
import pandas as pd
import os, numpy as np

In [84]:
path = r'C:\Users\Laptop88\Desktop\OneMount\brazilian-ecommerce'

In [85]:
list_files = os.listdir(path)
list_files.sort(reverse = False)
list_files

['olist_customers_dataset.csv',
 'olist_geolocation_dataset.csv',
 'olist_order_items_dataset.csv',
 'olist_order_payments_dataset.csv',
 'olist_order_reviews_dataset.csv',
 'olist_orders_dataset.csv',
 'olist_products_dataset.csv',
 'olist_sellers_dataset.csv',
 'product_category_name_translation.csv']

In [86]:
customers = pd.read_csv(path + '\\'+list_files[0])
geolocation = pd.read_csv(path + '\\'+list_files[1])
order_items = pd.read_csv(path + '\\'+list_files[2])
order_payments = pd.read_csv(path + '\\'+list_files[3])
order_reviews = pd.read_csv(path + '\\'+list_files[4])
orders = pd.read_csv(path + '\\'+list_files[5])
products = pd.read_csv(path + '\\'+list_files[6])
sellers = pd.read_csv(path + '\\'+list_files[7])
product_category_name = pd.read_csv(path + '\\'+list_files[8])

In [106]:
# Tính quãng đường theo kinh độ vĩ độ
def haversine_vectorize(lon1, lat1, lon2, lat2):
    """Returns distance, in kilometers, between one set of longitude/latitude coordinates and another"""
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
 
    newlon = lon2 - lon1
    newlat = lat2 - lat1
 
    haver_formula = np.sin(newlat/2.0)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(newlon/2.0)**2
 
    dist = 2 * np.arcsin(np.sqrt(haver_formula ))
    km = 6367 * dist #6367 for distance in KM for miles use 3958
    return km

In [137]:
# Tính các bước thời gian
orders['order_purchase_approved'] = (orders['order_approved_at'].astype(np.datetime64) - orders['order_purchase_timestamp'].astype(np.datetime64))/np.timedelta64(1,'h')
orders['order_approved_delivered_carrier'] = (orders['order_delivered_carrier_date'].astype(np.datetime64) - orders['order_approved_at'].astype(np.datetime64))/np.timedelta64(1,'h')
# Tính thời gian từ lúc giao cho vận chuyển đến khi giao thành công
estimated_delivery = (orders['order_estimated_delivery_date'].astype(np.datetime64) - orders['order_delivered_carrier_date'].astype(np.datetime64))/np.timedelta64(1,'h')
real_delivery = (orders['order_delivered_customer_date'].astype(np.datetime64) - orders['order_delivered_carrier_date'].astype(np.datetime64))/np.timedelta64(1,'h')
orders['order_carrier_deliveried'] = np.where(orders['order_delivered_customer_date'].isnull(), estimated_delivery, real_delivery)
orders['total_times'] = np.where((orders['order_approved_delivered_carrier']>0) & (orders['order_carrier_deliveried']>0),orders['order_purchase_approved']+orders['order_approved_delivered_carrier']+orders['order_carrier_deliveried'],np.nan)
# Loại bỏ đơn hàng mà thời gian giao thành công sớm hơn thời gian đưa bên vận chuyển, hoặc thời gian giao vận chuyển lớn hơn thời gian đơn xác nhận
orders[(orders['order_approved_delivered_carrier']>0) & (orders['order_carrier_deliveried']>0)].describe()

,order_purchase_approved,order_approved_delivered_carrier,order_carrier_deliveried,total_times
count,96245.000000,96245.000000,96245.000000,96245.000000
mean,9.666829,68.509998,228.303336,306.480162
std,19.275455,83.669261,212.976314,232.034150
min,0.000000,0.004167,0.000278,12.801944
25%,0.214444,21.632222,99.343333,164.411111
50%,0.337778,44.418056,171.495278,248.642222
75%,13.570278,86.909722,292.768056,384.142222
max,784.045556,3018.301667,4924.583333,5031.086667


* Bảng products

In [131]:
# Check null bảng products
products[products.isnull().any(axis=1)]
# Fill trường null cột category_name bằng "others"
products['product_category_name'] = products['product_category_name'].fillna('Others')
# Tạo bảng danh mục ID sản phẩm - mở file excel thêm trường thông tin Category để phân loại thêm
# Lấy thông tin tên tiếng anh từ bảng category cho bảng products
product_name = pd.merge(products,product_category_name, how = 'left', on = 'product_category_name')
# Check null value bảng product_name
product_name[product_name.isnull().any(axis=1)]


,product_id,product_category_name,product_name_lenght,product_description_lenght,product_photos_qty,product_weight_g,product_length_cm,product_height_cm,product_width_cm,product_category_name_english,Category
105,a41e356c76fab66334f36de622ecbd3a,Others,NaN,NaN,NaN,650.0,17.0,14.0,12.0,Others,Others
128,d8dee61c2034d6d075997acef1870e9b,Others,NaN,NaN,NaN,300.0,16.0,7.0,20.0,Others,Others
145,56139431d72cd51f19eb9f7dae4d1617,Others,NaN,NaN,NaN,200.0,20.0,20.0,20.0,Others,Others
154,46b48281eb6d663ced748f324108c733,Others,NaN,NaN,NaN,18500.0,41.0,30.0,41.0,Others,Others
197,5fb61f482620cb672f5e586bb132eae9,Others,NaN,NaN,NaN,300.0,35.0,7.0,12.0,Others,Others
...,...,...,...,...,...,...,...,...,...,...,...
32515,b0a0c5dd78e644373b199380612c350a,Others,NaN,NaN,NaN,1800.0,30.0,20.0,70.0,Others,Others
32589,10dbe0fbaa2c505123c17fdc34a63c56,Others,NaN,NaN,NaN,800.0,30.0,10.0,23.0,Others,Others
32616,bd2ada37b58ae94cc838b9c0569fecd8,Others,NaN,NaN,NaN,200.0,21.0,8.0,16.0,Others,Others
32772,fa51e914046aab32764c41356b9d4ea4,Others,NaN,NaN,NaN,1300.0,45.0,16.0,45.0,Others,Others


* Bảng customers, geolocation, sellers, orders, order_items, order_reviews, order_payments

In [132]:
# Check null bảng customers
customers[customers.isnull().any(axis=1)]
# Check unique
customers[['customer_id']].value_counts(ascending= False)
# Tương tự với các bảng còn lại

customer_id                     
00012a2ce6f8dcda20d059ce98491703    1
aa62a16b10e3fb24ecdefd7e4dc3fa65    1
aa738cdb72af39e38482fa25b992c67c    1
aa7210736aad65a84e964cfa8d64aa66    1
aa72002446dfbac0691304465c1b982d    1
                                   ..
5591abc9e7f9c644e7e7912aac65b718    1
5590e685a78102fb3ac27ecf07412750    1
558ea29f4c4dfd014345b6d3309549a2    1
558de2f6863bf7588338479d614522a3    1
ffffe8b65bbe3087b653a978c870db99    1
Length: 99441, dtype: int64

In [133]:
# Riêng bảng geolocation không unique theo "geolocation_zip_code_prefix", nên phương pháp lấy giá trị trung bình kinh độ và vĩ độ
# Groupby và tính trung bình tọa độ
geo_group = geolocation.groupby(['geolocation_zip_code_prefix','geolocation_city','geolocation_state']).mean().reset_index()
# Check unique cột "geolocation_zip_code_prefix"
geo_group[['geolocation_zip_code_prefix']].value_counts()
# Lấy giá trị min theo text do geolocation_city có giá trị trùng lặp chỉ khác nhau về vài kí tự dấu, và geolocation_state bị khác nhau mặc dù cùng 1 geolocation_city
geo_group = geo_group.groupby(['geolocation_zip_code_prefix'],sort=False).agg({'geolocation_city':'min','geolocation_state':'min','geolocation_lat':'mean','geolocation_lng':'mean'}).reset_index().reindex(columns=geo_group.columns)
#  Check lại unique của "geolocation_zip_code_prefix"
geo_group[['geolocation_zip_code_prefix']].value_counts()
geo_group.head()

,geolocation_zip_code_prefix,geolocation_city,geolocation_state,geolocation_lat,geolocation_lng
0,1001,sao paulo,SP,-23.550106,-46.634039
1,1002,sao paulo,SP,-23.546540,-46.634154
2,1003,sao paulo,SP,-23.549036,-46.635325
3,1004,sao paulo,SP,-23.549660,-46.634600
4,1005,sao paulo,SP,-23.549650,-46.635971


In [136]:
# Check mối quan hệ 2 bảng orders và order_items
orders['order_id'].value_counts(dropna=False, ascending=False) 
order_items['order_id'].value_counts(dropna=False, ascending=False)
# Merge bảng orders và bảng order_items bằng cột order_id
orders_items = pd.merge(orders,order_items,how = 'right',on = 'order_id')
order_item_product = pd.merge(orders_items,product_name,how = 'left',on = 'product_id')
# Merge bảng order_item_product và bảng sellers bằng cột seller_id
order_item_product_seller = pd.merge(order_item_product,sellers,how = 'left', on = 'seller_id')
order_seller_customer = pd.merge(order_item_product_seller,customers, how = 'left', on = 'customer_id')
# Tính khoảng cách giữa sellers và customers
order_seller_customer['sell_lat'] = pd.merge(order_seller_customer,geo_group,how='left',left_on='seller_zip_code_prefix',right_on='geolocation_zip_code_prefix')['geolocation_lat']
order_seller_customer['sell_lng'] = pd.merge(order_seller_customer,geo_group,how='left',left_on='seller_zip_code_prefix',right_on='geolocation_zip_code_prefix')['geolocation_lng']
order_seller_customer['cus_lat'] = pd.merge(order_seller_customer,geo_group,how='left',left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix')['geolocation_lat']
order_seller_customer['cus_lng'] = pd.merge(order_seller_customer,geo_group,how='left',left_on='customer_zip_code_prefix',right_on='geolocation_zip_code_prefix')['geolocation_lng']
order_seller_customer['dist'] = haversine_vectorize(order_seller_customer.sell_lat, order_seller_customer.sell_lng, 
                                                     order_seller_customer.cus_lat, order_seller_customer.cus_lng)
order_seller_customer['avg_time_per_dist'] = order_seller_customer['total_times']/order_seller_customer['dist']
order_seller_customer[order_seller_customer['avg_time_per_dist'].isnull()].head()


# order_seller_customer.columns

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_purchase_approved,order_approved_delivered_carrier,...,customer_unique_id,customer_zip_code_prefix,customer_city,customer_state,sell_lat,sell_lng,cus_lat,cus_lng,dist,avg_time_per_dist
7,000576fe39319847cbb9d288c5617fa6,9ed5e522dd9dd85b4af4a077526d8117,delivered,2018-07-04 12:08:27,2018-07-05 16:35:48,2018-07-05 12:15:00,2018-07-09 14:04:07,2018-07-25 00:00:00,28.455833,-4.346667,...,fda4476abb6307ab3c415b7e6d026526,11702,praia grande,SP,-22.122735,-51.386369,-24.014395,-46.438358,566.886034,NaN
56,002175704e8b209f61b9ad5cfd92b60e,a562db3c7cb9a68947debd30879b491e,delivered,2018-04-22 12:13:25,2018-04-24 17:24:48,2018-04-23 19:03:19,2018-05-02 20:38:44,2018-05-14 00:00:00,53.189722,-22.358056,...,e7d41c19da317af0612170197303d5a9,14802,araraquara,SP,-23.568994,-46.522437,-21.772226,-48.180222,228.538951,NaN
67,002834535f7a609a5c68266f173fa59e,89a6b1bfbe1b366503995b8df4c49450,delivered,2018-07-23 17:26:00,2018-07-28 23:30:59,2018-07-24 15:53:00,2018-08-11 01:48:33,2018-08-14 00:00:00,126.083056,-103.633056,...,457626c51309d814c33966c88a161545,52041,recife,PE,-23.625520,-46.611898,-8.034194,-34.894814,1842.281348,NaN
84,00310b0c75bb13015ec4d82d341865a4,0dad07848c618cc5a4679a1bfe1db8d2,canceled,2018-08-15 14:29:08,2018-08-15 15:04:25,NaN,NaN,2018-08-29 00:00:00,0.588056,NaN,...,ec979208947bbba310f2ad8e50963725,31160,belo horizonte,MG,-23.525769,-46.691768,-19.878923,-43.931505,418.606384,NaN
118,004345d16a1ab2c21962992c721c8643,3b723b649feb48f3cfbb9c728efb3d7d,delivered,2018-07-04 06:45:05,2018-07-05 16:09:18,2018-07-05 14:08:00,2018-07-11 17:26:37,2018-07-30 00:00:00,33.403611,-2.021667,...,8c07068e51d745035b296ebc1c8cd16a,72914,aguas lindas de goias,GO,-23.690940,-46.679675,-15.726302,-48.277261,623.643030,NaN


In [126]:
order_seller_customer.columns

Index(['order_id', 'customer_id', 'order_status', 'order_purchase_timestamp',
       'order_approved_at', 'order_delivered_carrier_date',
       'order_delivered_customer_date', 'order_estimated_delivery_date',
       'order_purchase_approved', 'order_approved_delivered_carrier',
       'order_carrier_deliveried', 'order_item_id', 'product_id', 'seller_id',
       'shipping_limit_date', 'price', 'freight_value',
       'product_category_name', 'product_name_lenght',
       'product_description_lenght', 'product_photos_qty', 'product_weight_g',
       'product_length_cm', 'product_height_cm', 'product_width_cm',
       'product_category_name_english', 'Category', 'seller_zip_code_prefix',
       'seller_city', 'seller_state', 'customer_unique_id',
       'customer_zip_code_prefix', 'customer_city', 'customer_state',
       'sell_lat', 'sell_lng', 'cus_lat', 'cus_lng', 'dist'],
      dtype='object')

In [96]:
order_items.head()

,order_id,order_item_id,product_id,seller_id,shipping_limit_date,price,freight_value
0,00010242fe8c5a6d1ba2dd792cb16214,1,4244733e06e7ecb4970a6e2683c13e61,48436dade18ac8b2bce089ec2a041202,2017-09-19 09:45:35,58.90,13.29
1,00018f77f2f0320c557190d7a144bdd3,1,e5f2d52b802189ee658865ca93d83a8f,dd7ddc04e1b6c2c614352b383efe2d36,2017-05-03 11:05:13,239.90,19.93
2,000229ec398224ef6ca0657da4fc703e,1,c777355d18b72b67abbeef9df44fd0fd,5b51032eddd242adc84c38acab88f23d,2018-01-18 14:48:30,199.00,17.87
3,00024acbcdf0a6daa1e931b038114c75,1,7634da152a4610f1595efa32f14722fc,9d7a1d34a5052409006425275ba1c2b4,2018-08-15 10:10:18,12.99,12.79
4,00042b26cf59d7ce69dfabb4e55b4fd9,1,ac6c3623068f30de03045865e4e10089,df560393f3a51e74553ab94004ba5c87,2017-02-13 13:57:51,199.90,18.14


In [97]:
order_payments.head()

,order_id,payment_sequential,payment_type,payment_installments,payment_value
0,b81ef226f3fe1789b1e8b2acac839d17,1,credit_card,8,99.33
1,a9810da82917af2d9aefd1278f1dcfa0,1,credit_card,1,24.39
2,25e8ea4e93396b6fa0d3dd708e76c1bd,1,credit_card,1,65.71
3,ba78997921bbcdc1373bb41e913ab953,1,credit_card,8,107.78
4,42fdf880ba16b47b59251dd489d4441a,1,credit_card,2,128.45


In [98]:
order_reviews.head()
# Có đơn hàng không ghi nhận ID ở bảng order_items

,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,7bc2406110b926393aa56f80a40eba40,73fc7af87114b39712e6da79b0a377eb,4,NaN,NaN,2018-01-18 00:00:00,2018-01-18 21:46:59
1,80e641a11e56f04c1ad469d5645fdfde,a548910a1c6147796b98fdf73dbeba33,5,NaN,NaN,2018-03-10 00:00:00,2018-03-11 03:05:13
2,228ce5500dc1d8e020d8d1322874b6f0,f9e4b658b201a9f2ecdecbb34bed034b,5,NaN,NaN,2018-02-17 00:00:00,2018-02-18 14:36:24
3,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
4,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53


In [99]:
order_reviews['review_answer_timestamp'] = pd.to_datetime(order_reviews['review_answer_timestamp'].astype(np.datetime64))
order_reviews['review_creation_date'] = pd.to_datetime(order_reviews['review_creation_date'].astype(np.datetime64))
order_reviews.head()
order_reviews.to_csv('order_reviews.csv', index=False)

In [100]:
orders.head()
# Đơn hàng đặt không có thông tin sản phẩm, phải mapping từ order_items

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date,order_purchase_approved,order_approved_delivered_carrier,order_carrier_deliveried
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18 00:00:00,0.178333,56.795833,145.503611
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13 00:00:00,30.713889,11.109167,288.945833
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04 00:00:00,0.276111,4.910278,220.274722
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15 00:00:00,0.298056,89.900000,226.811944
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26 00:00:00,1.030556,21.434722,46.507778


In [101]:
sellers.head()

,seller_id,seller_zip_code_prefix,seller_city,seller_state
0,3442f8959a84dea7ee197c632cb2df15,13023,campinas,SP
1,d1b65fc7debc3361ea86b5f14c68d2e2,13844,mogi guacu,SP
2,ce3ad9de960102d0677a81f5d0bb7b2d,20031,rio de janeiro,RJ
3,c0f3eea2e14555b6faeea3dd58c1b1c3,4195,sao paulo,SP
4,51a04a8a6bdcb23deccc82b0b80742cf,12914,braganca paulista,SP
